In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
alt.data_transformers.enable('json')
from sublimpy import tidy, utils
import pytz

In [3]:
seconds_in_timestep = 60*30
from metpy.constants import density_water

# Open datasets

In [4]:
# Open SOS Measurement Dataset
################################################
start_date = '20221130'
end_date = '20230509'
# open files
tidy_df = pd.read_parquet(f'tidy_df_{start_date}_{end_date}_noplanar_fit_clean.parquet')
# convert time column to datetime
tidy_df['time'] = pd.to_datetime(tidy_df['time'])
# limit data to our dates of interest, based on continuous snow cover at Kettle Ponds
tidy_df = tidy_df.set_index('time').sort_index().loc[start_date:end_date].reset_index()

tidy_df = utils.modify_df_timezone(tidy_df, pytz.UTC, 'US/Mountain')


# Open Turbpy Model Ensemble Dataset
################################################
model_df = pd.read_parquet("model_results.parquet")
# add a bunch of columns that are descriptive, from the config column which has multiple bits of info
model_df['z0'] = model_df['config'].apply(
    lambda v: v.split(' ')[-1]
)
model_df['e_sat_curve'] = model_df['config'].apply(
    lambda v: 'metpy' if 'metpy' in v else 'alduchov'
)
model_df['surface_measurement'] = model_df['config'].apply(
    lambda v: v.split(' ')[-3]
)
model_df['scheme'] = model_df['config'].apply(
    lambda v: 'andreas' if 'andreas lengths' in v else 'yang'
)
model_df['most_config'] = model_df['config'].apply(lambda s: ' '.join(s.split(' ')[:-3]))
# remove the scalar roughness length parameterization info 
model_df['most_config'] = model_df['most_config'].str.replace(' andreas lengths', '')
model_df['latent heat flux (mm)'] = -model_df['latent heat flux']*seconds_in_timestep/density_water/2838

model_df = utils.modify_df_timezone(model_df, pytz.UTC, 'US/Mountain')


# Open COARE model results
################################################
coare_model_results = pd.read_parquet("coare_model_results.parquet").reset_index()
coare_model_results['z0'] = coare_model_results.config.str.split(' ').apply(lambda x: x[-1])
coare_model_results['surface_measurement'] = coare_model_results.config.str.split(' ').apply(lambda x: x[0])
coare_model_results['e_sat_curve'] = coare_model_results.config.str.split(' ').apply(lambda x: x[1])

coare_model_results['hlb_mm'] = coare_model_results['hlb_gperm2s']*seconds_in_timestep/density_water

coare_model_results = utils.modify_df_timezone(coare_model_results, pytz.UTC, 'US/Mountain')

# Analysis

In [5]:
tidy_df = tidy.tidy_df_add_variable(
    tidy_df,
    (
        tidy_df.query("variable == 'SF_avg_1m_ue'")['value'].values + 
        tidy_df.query("variable == 'SF_avg_2m_ue'")['value'].values
    ), 
    'SF_avg_ue',
    'snow flux',
    1,
    'ue',
)


# Define times for different categories

In [6]:
bs_times = tidy_df.query("variable == 'SF_avg_ue'").query("value > 0").time
nobs_times = tidy_df.query("variable == 'SF_avg_ue'").query("value == 0").time
decoupled_times = tidy_df.query("variable == 'omega_3m_c'").query("value < 0.43").time
weaklycoupled_times = tidy_df.query("variable == 'omega_3m_c'").query("value >= 0.43").query("value <= 0.61").time
coupled_times = tidy_df.query("variable == 'omega_3m_c'").query("value > 0.61").time

In [7]:
lhflux_measurements_df = tidy_df[tidy_df.variable.isin([
    'w_h2o__1m_c',
    'w_h2o__2m_c',
    'w_h2o__3m_c',
    'w_h2o__5m_c',
    'w_h2o__10m_c',
    'w_h2o__15m_c',
    'w_h2o__20m_c',
])]
lhflux_measurements_df

,time,variable,value,height,tower,measurement
140,2022-11-29 17:00:00,w_h2o__2m_c,0.001048,2.0,c,w_h2o_
340,2022-11-29 17:00:00,w_h2o__3m_c,0.001886,3.0,c,w_h2o_
667,2022-11-29 17:00:00,w_h2o__20m_c,0.008915,20.0,c,w_h2o_
669,2022-11-29 17:00:00,w_h2o__15m_c,0.006543,15.0,c,w_h2o_
671,2022-11-29 17:00:00,w_h2o__10m_c,0.010222,10.0,c,w_h2o_
...,...,...,...,...,...,...
5239507,2023-05-09 17:30:00,w_h2o__3m_c,0.003844,3.0,c,w_h2o_
5239508,2023-05-09 17:30:00,w_h2o__5m_c,0.002175,5.0,c,w_h2o_
5239510,2023-05-09 17:30:00,w_h2o__10m_c,0.006808,10.0,c,w_h2o_
5239512,2023-05-09 17:30:00,w_h2o__15m_c,NaN,15.0,c,w_h2o_


In [8]:
bs_lhflux_df = lhflux_measurements_df[lhflux_measurements_df.time.isin(bs_times)]
nobs_lhflux_df = lhflux_measurements_df[lhflux_measurements_df.time.isin(nobs_times)]
decoupled_df = lhflux_measurements_df[lhflux_measurements_df.time.isin(decoupled_times)]
weaklycoupled_df = lhflux_measurements_df[lhflux_measurements_df.time.isin(weaklycoupled_times)]
coupled_df = lhflux_measurements_df[lhflux_measurements_df.time.isin(coupled_times)]

## Plot mean lh flux vertical profiles for different conditions

In [9]:
(
    alt.Chart(bs_lhflux_df).mark_line(point=True).encode(
        alt.X("mean(value):Q").sort('-y').scale(domain=[0,0.01]).title("Mean w'q' (g/m^2/s)"),
        alt.Y("height:Q")
    ).properties(title="During blowing snow", width=150, height=150)
 |
    alt.Chart(nobs_lhflux_df).mark_line(point=True).encode(
        alt.X("mean(value):Q").sort('-y').scale(domain=[0,0.01]).title("Mean w'q' (g/m^2/s)"),
        alt.Y("height:Q")
    ).properties(title="During no blowing snow", width=150, height=150)
|
    alt.Chart(decoupled_df).mark_line(point=True).encode(
        alt.X("mean(value):Q").sort('-y').scale(domain=[0,0.01]).title("Mean w'q' (g/m^2/s)"),
        alt.Y("height:Q")
    ).properties(title="During decoupling", width=150, height=150)
|
    alt.Chart(weaklycoupled_df).mark_line(point=True).encode(
        alt.X("mean(value):Q").sort('-y').scale(domain=[0,0.01]).title("Mean w'q' (g/m^2/s)"),
        alt.Y("height:Q")
    ).properties(title="During weak coupling", width=150, height=150)
|
    alt.Chart(coupled_df).mark_line(point=True).encode(
        alt.X("mean(value):Q").sort('-y').scale(domain=[0,0.01]).title("Mean w'q' (g/m^2/s)"),
        alt.Y("height:Q")
    ).properties(title="During coupling", width=150, height=150)
).configure_point(
    size=100
).resolve_scale(y='shared', x='shared').properties(title='Mean latent heat flux profiles during different conditions')

alt.HConcatChart(...)

In [48]:
bs_and_coupled_df = lhflux_measurements_df[lhflux_measurements_df.time.isin(
    set(bs_times).union(set(coupled_times)) 
)]
nobs_and_coupled_df = lhflux_measurements_df[lhflux_measurements_df.time.isin(
    set(nobs_times).union(set(coupled_times)) 
)]
during_bs_and_coupling = alt.Chart(bs_and_coupled_df).transform_aggregate(
    mean = "mean(value)",
    groupby=['height']
).transform_calculate(
    low_bound = 'datum.mean - 0.2*datum.mean',
    up_bound = 'datum.mean + 0.2*datum.mean'
).mark_errorbar(thickness=2.5).encode(
    alt.X("low_bound:Q").title(""),
    alt.X2("up_bound:Q"),
    alt.Y("height:Q")
).properties(title="During BS and coupling", width=150, height=150) +\
    alt.Chart(bs_and_coupled_df).mark_line(point=True, color='black').encode(
    alt.X("mean(value):Q").sort('-y').scale(domain=[0,0.01]).title("w'q' (dots) ± 20% (bars) (g/m^2/s)"),
    alt.Y("height:Q")
).properties(title="During BS and coupling", width=150, height=150)

nobs_and_coupling = alt.Chart(nobs_and_coupled_df).transform_aggregate(
    mean = "mean(value)",
    groupby=['height']
).transform_calculate(
    low_bound = 'datum.mean - 0.2*datum.mean',
    up_bound = 'datum.mean + 0.2*datum.mean'
).mark_errorbar(thickness=2.5).encode(
    alt.X("low_bound:Q").title(""),
    alt.X2("up_bound:Q"),
    alt.Y("height:Q")
).properties(title="During no BS and coupling", width=150, height=150) +\
    alt.Chart(nobs_and_coupled_df).mark_line(point=True, color='black').encode(
    alt.X("mean(value):Q").sort('-y').scale(domain=[0,0.01]).title("w'q' (dots) ± 20% (bars) (g/m^2/s)"),
    alt.Y("height:Q")
).properties(title="During BS and coupling", width=150, height=150)

(during_bs_and_coupling | nobs_and_coupling).configure_point(
    size=50, color='black'
).resolve_scale(y='shared', x='shared')

alt.HConcatChart(...)

## Plot vertical profiles exploring the December wind event

Sensible heat

In [144]:
src = tidy_df[tidy_df.variable.isin([
    'w_tc__1m_c', 'w_tc__2m_c', 'w_tc__3m_c', 'w_tc__5m_c', 'w_tc__10m_c', 'w_tc__15m_c', 'w_tc__20m_c',
    'w_tc__1m_ue',  'w_tc__1m_uw',  'w_tc__1m_d',
    'w_tc__3m_ue',  'w_tc__3m_uw',  'w_tc__3m_d',
    'w_tc__10m_ue', 'w_tc__10m_uw', 'w_tc__10m_d',
])]
# Tower uw was buried during this event, so lets not look at that data
src = src.query("tower != 'uw'")
w_tc_prof_timeseries_chart = alt.Chart(
    src[
        src.time.dt.minute == 0
    ].set_index('time').loc[
        "2022-12-21 12": "2022-12-21 23"
    ].reset_index()
).mark_line(point=True).encode(
        alt.X("mean(value):Q").sort('-y').title("w'tc'"), #.scale(domain=[-0.25,0], clamp=True),
        alt.Y("height:Q"),
        alt.Color("tower:N"),
        # alt.Color("hours(time):O"),
        alt.Facet("time:T").header(labelFontSize=0).title(None),
    ).properties(width=75,height=75)

Potential temperature

In [145]:
src = tidy_df[tidy_df.measurement.isin([
    'surface potential temperature',
    'potential temperature'
])].query("tower == 'c'")
theta_prof_timeseries_chart = alt.Chart(
    src[
        src.time.dt.minute == 0
    ].set_index('time').loc[
        "2022-12-21 12": "2022-12-21 23"
    ].reset_index()
).mark_line(point=True).encode(
        alt.X("mean(value):Q").sort('-y').title("theta").scale(zero=False),
        alt.Y("height:Q"),
        # alt.Color("hours(time):O"),
        alt.Facet("time:T").header(labelFontSize=0).title(None),
        tooltip='height:Q'
    ).properties(width=75,height=75)

lh flux 

In [146]:
src = tidy_df[tidy_df.variable.isin([
    'w_h2o__1m_c', 'w_h2o__2m_c', 'w_h2o__3m_c', 'w_h2o__5m_c', 'w_h2o__10m_c', 'w_h2o__15m_c', 'w_h2o__20m_c',
    'w_h2o__1m_ue',  'w_h2o__1m_uw',  'w_h2o__1m_d',
    'w_h2o__3m_ue',  'w_h2o__3m_uw',  'w_h2o__3m_d',
    'w_h2o__10m_ue', 'w_h2o__10m_uw', 'w_h2o__10m_d',
])]
# Tower uw was buried during this event, so lets not look at that data
src = src.query("tower != 'uw'")
w_q_prof_timeseries_chart = alt.Chart(
    src[
        src.time.dt.minute == 0
    ].set_index('time').loc[
        "2022-12-21 12": "2022-12-21 23"
    ].reset_index()
).mark_line(point=True).encode(
        alt.X("mean(value):Q").sort('-y').scale(domain=[0, 0.05], clamp=True).title("w'q'"),
        alt.Y("height:Q"),
        alt.Color("tower:N"),
        # alt.Color("hours(time):O"),
        alt.Facet("time:T").header(format="%m/%d %H:%M")
    ).properties(width=75,height=75)

RH

In [147]:
src = tidy_df[tidy_df.measurement=='RH'].query("height != 6").query("height != 12")
rh_prof_timeseries_chart = alt.Chart(
    src[
        src.time.dt.minute == 0
    ].set_index('time').loc[
        "2022-12-21 12": "2022-12-21 23"
    ].reset_index()
).mark_circle().encode(
        alt.X("mean(value):Q").sort('-y').title("RH (%)").scale(domain=[55,75]),
        alt.Y("height:Q"),
        alt.Color("tower:N"),
        # alt.Color("hours(time):O"),
        alt.Facet("time:T").header(labelFontSize=0).title(None),
        tooltip='height:Q'
    ).properties(width=75,height=75)

blowing snow time series

In [148]:
# src = tidy_df[tidy_df.variable == 'SF_avg_ue']
src = tidy_df[tidy_df.variable.isin(['SF_avg_1m_ue', 'SF_avg_2m_ue'])]
# src = src[src.time.isin(bs_times)]
bs_flux_timeseries_chart = alt.Chart(
    src.set_index('time').loc[
        "2022-12-21 12": "2022-12-21 23"
    ].reset_index().query("value > 0")
).mark_circle(point=True, size=75).encode(
        alt.X("time:T"),
        alt.Y("value:Q").scale(type='log').title("Blowing snow flux (g/m^2/s)"),
        alt.Color("height:O")
).properties(width=1140,height=100) 

src = tidy_df[tidy_df.variable.isin(['spd_3m_c', 'spd_5m_c', 'spd_20m_c'])]
windspd_timeseries_chart = alt.Chart(
    src.set_index('time').loc[
        "2022-12-21 12": "2022-12-21 23"
    ].reset_index().query("value > 0")
).mark_line().encode(
        alt.X("time:T").axis(labels=False).title(None),
        alt.Y("value:Q").title("Wind speed (m/s)"),
        alt.Color("height:O")
).properties(width=1140,height=100) 


src = tidy_df[tidy_df.variable.isin(['u*_3m_c', 'u*_5m_c', 'u*_20m_c'])]
ufric_timeseries_chart = alt.Chart(
    src.set_index('time').loc[
        "2022-12-21 12": "2022-12-21 23"
    ].reset_index().query("value > 0")
).mark_line().encode(
        alt.X("time:T").axis(labels=False).title(None),
        alt.Y("value:Q").title("Friction velocity (m/s)"),
        alt.Color("height:O")
).properties(width=1140,height=100) 


In [149]:
(
    ufric_timeseries_chart & windspd_timeseries_chart & bs_flux_timeseries_chart & 
    (w_q_prof_timeseries_chart & rh_prof_timeseries_chart & w_tc_prof_timeseries_chart & theta_prof_timeseries_chart)
).resolve_scale(color='independent')

alt.VConcatChart(...)

Look at time series of Snow surface temperature, and other related variables, to see what is going on with surface temperatures during the blowing snow event

In [150]:
src = tidy_df.set_index('time').sort_index().loc[
    "2022-12-21 12": "2022-12-22 23"
].reset_index()
# Create DF with renamed Tsnow variables to clarify that they are air sensors at this time because
# snowdepth during this time period is 30-40cm, so the in-snow T sensors are ~10, 20cm above snow
in_snow_sensors_as_air_sensors_df = src[src.variable.isin([
    'T_2m_c',
    'Tsnow_0_4m_d', 'Tsnow_0_5m_d', 'Tsnow_0_6m_d', 'Tsnow_0_7m_d',
    'SnowDepth_d'
])].pivot(index='time', columns='variable', values='value')
in_snow_sensors_as_air_sensors_df['T_00cm_c'] = in_snow_sensors_as_air_sensors_df['Tsnow_0_4m_d']
in_snow_sensors_as_air_sensors_df['T_10cm_c'] = in_snow_sensors_as_air_sensors_df['Tsnow_0_5m_d']
in_snow_sensors_as_air_sensors_df['T_20cm_c'] = in_snow_sensors_as_air_sensors_df['Tsnow_0_6m_d']
in_snow_sensors_as_air_sensors_df['T_30cm_c'] = in_snow_sensors_as_air_sensors_df['Tsnow_0_7m_d']
in_snow_sensors_as_air_sensors_df = in_snow_sensors_as_air_sensors_df.reset_index().melt(id_vars='time')   

# Create DF to hold the COARE model results we care about here
# and to rename the run we did using the in-snow temp
coare_model_results_local = coare_model_results[coare_model_results.config.isin([
    "Tsurf_d e_sat_alduchov 0.0005",
    "Tsurf_c e_sat_alduchov 0.0005",
    "Tsnow_0_4m_d e_sat_alduchov 0.0005",
])].set_index("time").sort_index().loc[
    src.time.min():src.time.max()
].reset_index()
coare_model_results_local.surface_measurement = coare_model_results_local.surface_measurement.replace(
    "Tsnow_0_4m_d",
    'T_00cm_c'
)

bs_case_study =(alt.Chart(src[src.variable == 'SF_avg_ue']).mark_line().encode(
    alt.X("time:T").axis(labels=False).title(None),
    alt.Y("value:Q").title("BS flux (g/m^2/s)"),
).properties(height = 100) &\
alt.Chart(in_snow_sensors_as_air_sensors_df[in_snow_sensors_as_air_sensors_df.variable.isin(
    ['T_00cm_c', 'T_10cm_c', 'T_20cm_c', 'T_30cm_c', 'T_2m_c']
)]).mark_line().encode(
    alt.X("time:T").axis(labels=False).title(None),
    alt.Y("value:Q").title("Temp. (˚C)"),
    alt.Color("variable:O").scale(
        domain = ['T_00cm_c', 'T_10cm_c', 'T_20cm_c', 'T_30cm_c', 'T_2m_c'],
        range=['lightsteelblue', 'cornflowerblue', 'royalblue', 'blue', 'black']
    ).title(["Temp. Measurement", "(near-surface and air)"]),
).resolve_scale(x='shared', y='independent', color='independent').properties(height = 100) &\
alt.Chart(src[src.variable.isin([
    'T_2m_c',
    'Tsurf_d',
    'Tsurf_c',
    'Tsurf_uw',
    'Tsurf_ue',
    'Tsurf_rad_d',
])]).mark_line().encode(
    alt.X("time:T").axis(labels=False).title(None),
    alt.Y("value:Q").title("Temp. (˚C)"),
    alt.Color("variable:O").scale(range=[
        'black', '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd'
    ]).title(["Temp. Measurement", "(surface and air)"]),
    detail='variable'
).resolve_scale(x='shared', y='independent').properties(height = 100)).resolve_scale(color='independent', strokeDash='independent') &\
alt.Chart(
    tidy_df.query("variable == 'w_h2o__3m_c'").set_index("time").loc[
        "20221221": "20221223"
    ].reset_index()
).mark_line(color='black').encode(
    alt.X("time:T"),
    alt.Y("value:Q"),
).properties(height = 100) + alt.Chart(
    coare_model_results_local
).mark_line().encode(
    alt.X("time:T"),
    alt.Y("hlb_gperm2s").title("w'q' (g/m^s/2)"),
    alt.Color("surface_measurement").title([
        "Tsurf used for",
        "model input"
    ])
)

In [151]:
src = tidy_df.set_index('time').sort_index().loc[
    "20230210": "20230212"
].reset_index()
# rename the Tsnow variables so clarify that the snowdepth during this time period is 88cm, 
# so the in-snow T sensors are actually ~10, 20, cm above the snow surface
# When we looked at T sensors at 80cm and 90cm, they were clearly under snow (signal attenuated)
# So we think its a decent estimate that the snow by the instrument is actually ~1m deep
in_snow_sensors_as_air_sensors_df = src[src.variable.isin([
    'T_2m_c',
    'Tsnow_1_0m_d',
    'Tsnow_1_1m_d',
    'Tsnow_1_2m_d',
    'Tsnow_1_3m_d',
    # 'Tsnow_1_4m_d',
    'SnowDepth_d'
])].pivot(index='time', columns='variable', values='value')
in_snow_sensors_as_air_sensors_df['T_00cm_c'] = in_snow_sensors_as_air_sensors_df['Tsnow_1_0m_d']
in_snow_sensors_as_air_sensors_df['T_10cm_c'] = in_snow_sensors_as_air_sensors_df['Tsnow_1_1m_d']
in_snow_sensors_as_air_sensors_df['T_20cm_c'] = in_snow_sensors_as_air_sensors_df['Tsnow_1_2m_d']
in_snow_sensors_as_air_sensors_df['T_30cm_c'] = in_snow_sensors_as_air_sensors_df['Tsnow_1_3m_d']
in_snow_sensors_as_air_sensors_df = in_snow_sensors_as_air_sensors_df.reset_index().melt(id_vars='time')

# Create DF to hold the COARE model results we care about here
# and to rename the run we did using the in-snow temp
coare_model_results_local = coare_model_results[coare_model_results.config.isin([
    "Tsurf_d e_sat_alduchov 0.0005",
    "Tsurf_c e_sat_alduchov 0.0005",
    "Tsnow_1_0m_d e_sat_alduchov 0.0005",
])].set_index("time").sort_index().loc[
    src.time.min():src.time.max()
].reset_index()
coare_model_results_local.surface_measurement = coare_model_results_local.surface_measurement.replace(
    'Tsnow_1_0m_d',
    'T_00cm_c'
)

clear_case_study = (alt.Chart(src[src.variable == 'SF_avg_ue']).mark_line().encode(
    alt.X("time:T").axis(labels=False).title(None),
    alt.Y("value:Q").title("BS flux (g/m^2/s)"),
).properties(height = 100) &\
alt.Chart(in_snow_sensors_as_air_sensors_df[in_snow_sensors_as_air_sensors_df.variable.isin(
    ['T_2m_c', 'T_00cm_c', 'T_10cm_c', 'T_20cm_c','T_30cm_c']
)]).mark_line().encode(
    alt.X("time:T").axis(labels=False).title(None),
    alt.Y("value:Q").title("Temp. (˚C)"),
    alt.Color("variable:O").scale(
        domain = ['T_00cm_c', 'T_10cm_c', 'T_20cm_c','T_30cm_c', 'T_2m_c', ],
        range=['lightsteelblue', 'cornflowerblue', 'royalblue', 'blue', 'black']
    ).title(["Temp. Measurement", "(near-surface and air)"]),
    # alt.StrokeDash("measurement:N"),
    detail='variable'
).resolve_scale(x='shared', y='independent', color='independent').properties(height = 100) &\
alt.Chart(src[src.variable.isin([
    'T_2m_c',
    'Tsurf_d',
    'Tsurf_c',
    'Tsurf_uw',
    'Tsurf_ue',
    'Tsurf_rad_d',
])]).mark_line().encode(
    alt.X("time:T").axis(labels=False).title(None),
    alt.Y("value:Q").title("Temp. (˚C)"),
    alt.Color("variable:O").scale(range=[
        'black', '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd'
    ]).title(["Temp. Measurement", "(surface and air)"]),
    # alt.StrokeDash("measurement:N"),
    detail='variable'
).resolve_scale(x='shared', y='independent').properties(height = 100)).resolve_scale(color='independent', strokeDash='independent') &\
alt.Chart(
    tidy_df.query("variable == 'w_h2o__3m_c'").set_index("time").loc[
        "20230210": "20230212"
    ].reset_index()
).mark_line(color='black').encode(
    alt.X("time:T"),
    alt.Y("value:Q")
).properties(height = 100) + alt.Chart(
    coare_model_results_local
).mark_line().encode(
    alt.X("time:T"),
    alt.Y("hlb_gperm2s").title("w'q' (g/m^s/2)"),
    alt.Color("surface_measurement:N").title([
        "Tsurf used for",
        "model input"
    ])
)

In [152]:
(bs_case_study | clear_case_study)

alt.HConcatChart(...)